<a href="https://colab.research.google.com/github/maryamataei1995/01052023/blob/main/workshop_pattern_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn import svm

# Data onboarding
## Load the data

In [72]:
df = pd.read_csv('ws1_data.csv',header=None)

In [73]:
df

,0,1,2,3,4
0,7.0,3.2,4.7,1.4,1
1,6.4,3.2,4.5,1.5,1
2,6.9,3.1,4.9,1.5,1
3,5.5,2.3,4.0,1.3,1
4,6.5,2.8,4.6,1.5,1
...,...,...,...,...,...
95,6.7,3.0,5.2,2.3,2
96,6.3,2.5,5.0,1.9,2
97,6.5,3.0,5.2,2.0,2
98,6.2,3.4,5.4,2.3,2


### Find out how the data oriented and determine the sizes and Visualise the data

In [74]:
df . describe()

,0,1,2,3,4
count,100.000000,100.000000,100.000000,100.000000,100.000000
mean,6.262000,2.872000,4.906000,1.676000,1.500000
std,0.662834,0.332751,0.825578,0.424769,0.502519
min,4.900000,2.000000,3.000000,1.000000,1.000000
25%,5.800000,2.700000,4.375000,1.300000,1.000000
50%,6.300000,2.900000,4.900000,1.600000,1.500000
75%,6.700000,3.025000,5.525000,2.000000,2.000000
max,7.900000,3.800000,6.900000,2.500000,2.000000


### Feature extraction if needed

In [75]:
df.corr()

,0,1,2,3,4
0,1.000000,0.553855,0.828479,0.593709,0.494305
1,0.553855,1.000000,0.519802,0.566203,0.308080
2,0.828479,0.519802,1.000000,0.823348,0.786424
3,0.593709,0.566203,0.823348,1.000000,0.828129
4,0.494305,0.308080,0.786424,0.828129,1.000000


### Data division

In [76]:
x= df.iloc[:,:-1]
y= df.iloc[:,-1]
x,y

(      0    1    2    3
 0   7.0  3.2  4.7  1.4
 1   6.4  3.2  4.5  1.5
 2   6.9  3.1  4.9  1.5
 3   5.5  2.3  4.0  1.3
 4   6.5  2.8  4.6  1.5
 ..  ...  ...  ...  ...
 95  6.7  3.0  5.2  2.3
 96  6.3  2.5  5.0  1.9
 97  6.5  3.0  5.2  2.0
 98  6.2  3.4  5.4  2.3
 99  5.9  3.0  5.1  1.8
 
 [100 rows x 4 columns],
 0     1
 1     1
 2     1
 3     1
 4     1
      ..
 95    2
 96    2
 97    2
 98    2
 99    2
 Name: 4, Length: 100, dtype: int64)

In [84]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.25)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(75, 4)
(75,)
(25, 4)
(25,)


## Feature selection

In [85]:
from sklearn.exceptions import FitFailedWarning
from itertools import combinations
FDR=[]
pairs = list(combinations(range(4),2))
for pairInd in range(len(pairs)):
  pair=pairs[pairInd]
  pair=list(pair)
  print(pair)
  mu1_pair = np.mean(X_train[y_train==1][pair], axis=0)
  mu2_pair = np.mean(X_train[y_train==2][pair], axis=0)
  sigma1_pair= np.cov(X_train[y_train==1][pair].T)
  sigma2_pair= np.cov(X_train[y_train==2][pair].T)
  FDRn_pair=np.dot(mu1_pair-mu2_pair,np.linalg.inv(sigma1_pair+sigma2_pair).dot(mu1_pair-mu2_pair))

  FDR.append(FDRn_pair)
  print(f"fdr: {FDRn_pair}")



[0, 1]
fdr: 0.5853494347486212
[0, 2]
fdr: 4.502440486588041
[0, 3]
fdr: 3.8588607284960577
[1, 2]
fdr: 3.3118894370015846
[1, 3]
fdr: 4.760344709804729
[2, 3]
fdr: 4.527257353178091


In [87]:
FDR

[0.5853494347486212,
 4.502440486588041,
 3.8588607284960577,
 3.3118894370015846,
 4.760344709804729,
 4.527257353178091]

In [88]:
Max=FDR.index(max(FDR))
print(Max)

4


In [91]:
pairs[Max]

(1, 3)

## Data normalisation

In [90]:
from sklearn import preprocessing
normalizer = preprocessing.StandardScaler()
X_train = normalizer.fit_transform(X_train)
X_test = normalizer.transform(X_test)

In [94]:
#Data modeling using the normal distribution
#Maximum likelihood estimation of the parameters and a priori
columns=pairs[Max]

XTrain=X_train[:,columns]
XTest=X_test[:,columns]
mu1 = np.mean(XTrain[y_train==1], axis=0)
print(mu1)
mu2 = np.mean(XTrain[y_train==2], axis=0)
sigma1= np.cov(XTrain[y_train==1].T)
sigma2= np.cov(XTrain[y_train==2].T)
mu = (mu1, mu2)
Sigma = (sigma1, sigma2)


[-0.39009673 -0.86752395]


In [95]:
print(mu)

(array([-0.39009673, -0.86752395]), array([0.34133464, 0.75908346]))


In [96]:
print(sigma1)

[[0.97097258 0.31215392]
 [0.31215392 0.22796581]]


In [97]:
n=len(y_train)
p=(np.sum(y_train==1)/n, np.sum(y_train==2)/n)
print(p)

(0.4666666666666667, 0.5333333333333333)


In [98]:
m=len(mu)
nTest= len(XTest)
g=np.zeros((25,2))
D=np.zeros((nTest,m))
for classInd in range(m):
  W = -0.5  * Sigma[classInd]
  w= np.dot(np.linalg.inv(Sigma[classInd]),mu[classInd])
  print(np.log(np.linalg.det(Sigma[classInd])))
  wi0= np.dot(-0.5 * mu[classInd].T,np.linalg.inv(Sigma[classInd]).dot (mu[classInd])) + (-0.5 *  np.log(np.linalg.det(Sigma[classInd]))) + np.log(p[classInd])
  for n in range(25):
    g[n,classInd]=np.dot(XTest[n,:].T,W).dot(XTest[n,:])+ np.dot(w.T,XTest[n,:]) + wi0
print("g=",g)
print(y_test)


-2.088212057115294
-1.5530803011708527
g= [[  2.84951543  -2.8262582 ]
 [ -2.91423034  -0.41207502]
 [ -3.26065513  -0.12204378]
 [  1.17275452  -1.92888894]
 [ -7.76002603   1.55929695]
 [  2.84951543  -2.8262582 ]
 [  3.38480054  -2.90890539]
 [ -1.07718959  -3.46957493]
 [  3.4813597   -3.49933378]
 [  1.74215824  -2.55592532]
 [  5.94591596  -4.94304039]
 [  1.56579693  -2.19273995]
 [ -9.14082838   1.99778924]
 [  2.84951543  -2.8262582 ]
 [ -9.81101691   1.13096798]
 [-11.43235888   2.91367592]
 [ -5.69956431   0.47724303]
 [ -5.47987392   0.3337127 ]
 [  3.14796884  -4.50544557]
 [ -8.05419118   1.49692514]
 [ -1.53976657  -0.89330142]
 [  3.83673369  -3.06198943]
 [  0.48833433  -2.52728422]
 [ -6.26646344  -4.81696443]
 [  2.23087835  -2.81404785]]
5     1
66    2
57    2
34    1
78    2
21    1
47    1
37    1
32    1
1     1
17    1
26    1
82    2
49    1
91    2
68    2
97    2
58    2
48    1
63    2
27    1
38    1
6     1
81    2
44    1
Name: 4, dtype: int64


In [101]:
ClassList=[]
for i in range(0,nTest-1):
  if g[i][0]>=g[i][1]:
    ClassList.append(f"{g[i][:]} is class 1")
  else:
    ClassList.append(f"{g[i][:]} is class 2")

for i in range(0,nTest-1):
  print(ClassList[i][:])

[ 2.84951543 -2.8262582 ] is class 1
[-2.91423034 -0.41207502] is class 2
[-3.26065513 -0.12204378] is class 2
[ 1.17275452 -1.92888894] is class 1
[-7.76002603  1.55929695] is class 2
[ 2.84951543 -2.8262582 ] is class 1
[ 3.38480054 -2.90890539] is class 1
[-1.07718959 -3.46957493] is class 1
[ 3.4813597  -3.49933378] is class 1
[ 1.74215824 -2.55592532] is class 1
[ 5.94591596 -4.94304039] is class 1
[ 1.56579693 -2.19273995] is class 1
[-9.14082838  1.99778924] is class 2
[ 2.84951543 -2.8262582 ] is class 1
[-9.81101691  1.13096798] is class 2
[-11.43235888   2.91367592] is class 2
[-5.69956431  0.47724303] is class 2
[-5.47987392  0.3337127 ] is class 2
[ 3.14796884 -4.50544557] is class 1
[-8.05419118  1.49692514] is class 2
[-1.53976657 -0.89330142] is class 2
[ 3.83673369 -3.06198943] is class 1
[ 0.48833433 -2.52728422] is class 1
[-6.26646344 -4.81696443] is class 2


##Performance evaluation

In [105]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
clf = LinearDiscriminantAnalysis()
clf.fit(XTrain, y_train)

# Make predictions on the test set
y_pred = clf.predict(XTest)

# Calculate the classification accuracy
accuracy = np.sum(y_pred == y_test) / len(y_test)

print('Classification accuracy:', accuracy)

Classification accuracy: 1.0
